<h1 align="center">Kaggle Decal #18</h1>
<h2 align="center"> Introduction to <a href="https://keras.io/">Keras</a></h2>


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, Dropout, MaxPooling2D
from keras.optimizers import SGD
import utils
import numpy as np
import matplotlib.pyplot as plt

## Load the data

In [ ]:
num_classes = 10
X, labels = utils.load_mnist_dataset()
Y = utils.one_hot(labels, num_classes)

In [ ]:
# split testing and training set
X_train, Y_train, X_test, Y_test = utils.train_test_split(X, Y, test_size=0.2)


## Initialize the model and Setup the layers

In [ ]:
model = Sequential()
model.add(Dense(output_dim=800, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))

## Compile and set Optimization

In [ ]:
# sgd = SGD(momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

## Train the model

In [ ]:
model.fit(X_train, Y_train, nb_epoch=5, batch_size=128)

## Evaluate the accuracy of the model

In [ ]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print("\nLoss {}, Accuracy {}".format(*loss_and_metrics))

## Make new predictions with the model

In [ ]:
classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)

In [ ]:
index=3
utils.plot_mnist(X_test[index], title='Prediction: {}'.format(classes[index]))


Now you may be wondering what examples caused the network to fail. And for a lot of them, you could expect to get them wrong as well!

In [ ]:
start = 19
for i in range(start, len(X_test)):
    actual = np.argmax(Y_test[i])
    predicted = classes[i]
    if predicted != actual:
        print("Model misclassified a {} as a {} at index {}".format(actual, predicted, i))
        utils.plot_mnist(X_test[i])
        break

## Let's Get some Convolution up in here
However, we'll need to get our data in a form that is friendly with convolutions. 

Let's keep using the MNIST dataset as a quick test. Each row in our dataset is a 784 length vector that represents the digit. Why don't we make this into a 28x28 matrix instead

In [ ]:
# reload in case we want to do this part separately
X_flat, labels = utils.load_mnist_dataset()
Y = utils.one_hot(labels)

In [ ]:
X = X_flat.reshape((len(X_flat),28, 28,1))
X_train, Y_train, X_test, Y_test = utils.train_test_split(X, Y, test_size=0.2)

In [ ]:
nb_filters=32
kernel_size = (3,3)
input_shape = (28, 28, 1)
pool_size = (2, 2)
nb_classes = 10

In [ ]:
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, 2, 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, nb_epoch=12, batch_size=128, validation_data=(X_test, Y_test))

In [ ]:
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)
print("\nLoss {}, Accuracy {}".format(*loss_and_metrics))